In [19]:
%autoreload
# %autoreload
import mcs
from scipy import sparse
import numpy as np

c = [1]
A_ineq = sparse.csr_matrix([1])
b_ineq = [1.5]
A_eq = sparse.csr_matrix([1])
b_eq = [1]
lb = [-np.inf]
ub = [ np.inf]

lp_feas = mcs.MILP_LP(c=c, A_ineq=A_ineq,b_ineq=b_ineq,A_eq=A_eq,b_eq=b_eq,lb=lb,ub=ub,solver='scip')
opt = lp_feas.slim_solve()
x, min_cx, status = lp_feas.solve()

lp_unbd = mcs.MILP_LP(c=c, A_ineq=A_ineq,b_ineq=b_ineq,lb=lb,ub=ub,solver='scip')
opt = lp_unbd.slim_solve()
x, min_cx, status = lp_unbd.solve()

c = [0]
A_ineq = sparse.csr_matrix([-1])
b_ineq = [-1.5]
lp_noopt = mcs.MILP_LP(c=c, A_ineq=A_ineq,b_ineq=b_ineq,lb=lb,ub=ub,solver='scip')
opt = lp_noopt.slim_solve()
x, min_cx, status = lp_noopt.solve()

c = [-1]
A_ineq = sparse.csr_matrix([-1])
b_ineq = [-1.5]
lp_infeas = mcs.MILP_LP(c=c, A_ineq=A_ineq,b_ineq=b_ineq,A_eq=A_eq,b_eq=b_eq,lb=lb,ub=ub,solver='scip')
opt = lp_infeas.slim_solve()
x, min_cx, status = lp_infeas.solve()

In [1]:
%load_ext autoreload
from scipy import sparse
import numpy as np
from importlib import reload
import mcs

In [12]:
# example setup (default: minimize)
# main problem
c = [-1,0,0,0]
A_ineq = sparse.csr_matrix([[1,1,0,0],[-1,0,0,0]])
b_ineq = [3,-1]
A_eq = sparse.csr_matrix([[0,1,0,0]])
b_eq = [0.6]
lb = [1,0,0,0]
ub = [np.inf, np.inf, 1, 1]
vtype = 'CCBB'
# indicator constraints
ic_binv = [2,3]
ic_A = sparse.csr_matrix([[1,1,0,0],[1,0,0,0]])
ic_b = [0,1.2]
ic_sense = "LE"
ic_indicval = [1,0]
indic_constr = mcs.Indicator_constraints(ic_binv, ic_A, ic_b, ic_sense, ic_indicval)

In [13]:
%autoreload
my_prob = mcs.MILP_LP(  c=c,
                        A_ineq=A_ineq,
                        b_ineq=b_ineq,
                        A_eq=A_eq,
                        b_eq=b_eq,
                        lb=lb,
                        ub=ub,
                        vtype=vtype, 
                        indic_constr=indic_constr,
                        solver='scip')
x, opt_cx, status = my_prob.solve()
# solution.get_status() returns an integer code
print("Solution status = ", str(status))
# the following line prints the corresponding string
print("Solution value  = ", str(round(opt_cx,5)))
sol = {'x'+str(i) : round(x[i],5) for i in range(len(x))}
print(sol)

presolving:
(round 1, fast)       3 del vars, 4 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 2 impls, 0 clqs
(round 2, fast)       4 del vars, 7 del conss, 0 add conss, 6 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 2 impls, 0 clqs
(round 3, fast)       4 del vars, 7 del conss, 0 add conss, 12 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 2 impls, 0 clqs
(round 4, fast)       7 del vars, 10 del conss, 0 add conss, 12 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 2 impls, 0 clqs
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
presolving (5 rounds: 5 fast, 1 medium, 1 exhaustive):
 8 deleted vars, 10 deleted constraints, 0 added constraints, 12 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 2 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal s

In [53]:
%load_ext autoreload
import cobra
from importlib import reload
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
iml1515 = cobra.io.read_sbml_model("iML1515.sbml")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
%load_ext autoreload
import cobra
import numpy as np
from scipy import sparse
from mcs import compress_model
def remove_ext_mets(network):
    external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
    network.remove_metabolites(external_mets)
    S = cobra.util.create_stoichiometric_matrix(network)
    obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
    network.remove_reactions(obsolete_reacs)
    return network


In [6]:
%autoreload
import efmtool_link.efmtool4cobra as efm
from mcs import fba,compress_model,compress_model_parallel
efmlrs = cobra.io.read_sbml_model("efmlrs.sbml")
print(fba(efmlrs))
print(len(efmlrs.reactions))
a = compress_model(efmlrs,['r02','r08'])
print(fba(efmlrs))
print(len(efmlrs.reactions))
b = compress_model_parallel(efmlrs, ['r02','r08'])
print(fba(efmlrs))
print(len(efmlrs.reactions))
c = compress_model(efmlrs,['r02','r08'])
print(fba(efmlrs))
print(len(efmlrs.reactions))
d = compress_model_parallel(efmlrs, ['r02','r08'])
print(fba(efmlrs))
print(len(efmlrs.reactions))
e = compress_model(efmlrs,['r02','r08'])
print(fba(efmlrs))
print(len(efmlrs.reactions))
# a = efm.compress_model_sympy(efmlrs)

# compress_model(efmlrs,['Growth','EthEx'])

<Solution 15.000 at 0x18e0aa18ac8>
22
<Solution 15.000 at 0x18e0a96bfc8>
10
<Solution 15.000 at 0x18e0a631988>
9
<Solution 15.000 at 0x18e0aa96088>
8
<Solution 15.000 at 0x18e0a9a8948>
7
<Solution 15.000 at 0x18e0a984448>
7


In [8]:
%autoreload
import efmtool_link.efmtool4cobra as efm
from mcs import fba,remove_blocked_reactions,compress_model,compress_model_parallel,stoichmat_coeff2rational,remove_conservation_relations
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")

bound_thres = 1000
for i in range(len(ecc2.reactions)):
    if ecc2.reactions[i].lower_bound <= -bound_thres:
        ecc2.reactions[i].lower_bound = -np.inf
    if ecc2.reactions[i].upper_bound >=  bound_thres:
        ecc2.reactions[i].upper_bound =  np.inf

remove_blocked_reactions(ecc2)
stoichmat_coeff2rational(ecc2)
remove_conservation_relations(ecc2)

print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))
a = compress_model(ecc2,['Growth','EthEx'])
print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))
b = compress_model_parallel(ecc2, ['Growth','EthEx'])
print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))
c = compress_model(ecc2,['Growth','EthEx'])
print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))
d = compress_model_parallel(ecc2, ['Growth','EthEx'])
print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))
e = compress_model(ecc2, ['Growth','EthEx'])
print(fba(ecc2,solver='cplex'))
print(len(ecc2.reactions))

<Solution 2.375 at 0x2127f44b7c8>
122
<Solution 2.375 at 0x2127f446c48>
70
<Solution 2.375 at 0x2127f9dee08>
70
<Solution 2.375 at 0x2121e386d08>
70
<Solution 2.375 at 0x2127f4268c8>
70
<Solution 2.375 at 0x2121e3c8e08>
70


In [4]:
%autoreload
import efmtool_link.efmtool4cobra as efm
import time
from mcs import fba,compress_model,compress_model_parallel
ecc2 = cobra.io.read_sbml_model("ECC2.sbml")
external_mets = [i for i,cpts in zip(ecc2.metabolites,ecc2.metabolites.list_attr("compartment")) if cpts == 'External_Species']
ecc2.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(ecc2)
obsolete_reacs = [reac for reac,b_rempty in zip(ecc2.reactions,np.any(S,0)) if not b_rempty]
ecc2.remove_reactions(obsolete_reacs)

bound_thres = 1000
for i in range(len(ecc2.reactions)):
    if ecc2.reactions[i].lower_bound <= -bound_thres:
        ecc2.reactions[i].lower_bound = -np.inf
    if ecc2.reactions[i].upper_bound >=  bound_thres:
        ecc2.reactions[i].upper_bound =  np.inf


print(fba(ecc2))
print(len(ecc2.reactions))
tic = time.time()
a = compress_model(ecc2,['Ec_biomass_iJO1366_core_53p95M','EX_etoh_e','EX_ac_e'])
print(fba(ecc2))
print(len(ecc2.reactions))
print(time.time() - tic)
tic = time.time()
b = compress_model_parallel(ecc2, ['Ec_biomass_iJO1366_core_53p95M','EX_etoh_e','EX_ac_e'])
print(fba(ecc2))
print(len(ecc2.reactions))
print(time.time() - tic)
tic = time.time()
c = compress_model(ecc2,['Ec_biomass_iJO1366_core_53p95M','EX_etoh_e','EX_ac_e'])
print(fba(ecc2))
print(len(ecc2.reactions))
print(time.time() - tic)
tic = time.time()
d = compress_model_parallel(ecc2, ['Ec_biomass_iJO1366_core_53p95M','EX_etoh_e','EX_ac_e'])
print(fba(ecc2))
print(len(ecc2.reactions))
print(time.time() - tic)
tic = time.time()
e = compress_model(ecc2, ['Ec_biomass_iJO1366_core_53p95M','EX_etoh_e','EX_ac_e'])
print(fba(ecc2))
print(len(ecc2.reactions))
print(time.time() - tic)

<Solution 0.982 at 0x2127f9afbc8>
499


AttributeError: 'float' object has no attribute 'p'

In [3]:
%autoreload
import efmtool_link.efmtool4cobra as efm
import time
from mcs import fba,compress_model,remove_blocked_reactions,compress_model_parallel,stoichmat_coeff2rational,remove_conservation_relations
iml = cobra.io.read_sbml_model("iML1515.sbml")

bound_thres = 1000
for i in range(len(iml.reactions)):
    if iml.reactions[i].lower_bound <= -bound_thres:
        iml.reactions[i].lower_bound = -np.inf
    if iml.reactions[i].upper_bound >=  bound_thres:
        iml.reactions[i].upper_bound =  np.inf

remove_blocked_reactions(iml)
stoichmat_coeff2rational(iml)
remove_conservation_relations(iml)

# BIOMASS_Ec_iML1515_core_75p37M
print(fba(iml))
print(len(iml.reactions))
tic = time.time()
a = compress_model(iml,['EX_etoh_e','EX_ac_e'])
print(fba(iml))
print(len(iml.reactions))
print(time.time() - tic)
tic = time.time()
b = compress_model_parallel(iml, ['EX_etoh_e','EX_ac_e'])
print(fba(iml))
print(len(iml.reactions))
print(time.time() - tic)
tic = time.time()
c = compress_model(iml,['EX_etoh_e','EX_ac_e'])
print(fba(iml))
print(len(iml.reactions))
print(time.time() - tic)
tic = time.time()
d = compress_model_parallel(iml, ['EX_etoh_e','EX_ac_e'])
print(fba(iml))
print(len(iml.reactions))
print(time.time() - tic)
tic = time.time()
e = compress_model(iml, ['BIOMASS_Ec_iML1515_core_75p37M','EX_etoh_e','EX_ac_e'])
print(fba(iml))
print(len(iml.reactions))
print(time.time() - tic)

<Solution 0.877 at 0x2121de2d8c8>
2711
<Solution 0.877 at 0x2127fad6048>
1246
287.4889974594116
<Solution 0.877 at 0x2121e326308>
1227
7.871450662612915
<Solution 0.877 at 0x2121e046c08>
1218
15.99288296699524
<Solution 0.877 at 0x2127fb056c8>
1215
8.681232690811157


KeyError: 'BIOMASS_Ec_iML1515_core_75p37M'

In [8]:
%autoreload
b = compress_model_parallel(iml, ['BIOMASS_Ec_iML1515_core_75p37M','EX_etoh_e','EX_ac_e'])


TypeError: unhashable type: 'list'

In [5]:
%autoreload
import mcs
from mcs import fva,fba
import cProfile
reload(mcs)
# opt = ecc2.optimize()
# print(opt)
# cProfile.runctx("fba(iml1515,solver='scip')",globals(),locals())
# opt2 = fba(iml1515,solver='scip')
# print(opt2)
# opt3 = cobra.flux_analysis.variability.flux_variability_analysis(iml1515)
# print(opt3)
# cProfile.runctx("mcs.fva(iml1515)",globals(),locals())
opt4 = fva(iml1515, solver='cplex')
print(opt4)
# cProfile.runctx()

           minimum     maximum
CYTDK2    0.000000  228.140000
XPPT      0.000000  114.070000
HXPRT     0.000000  114.070000
NDPK5   -76.922791  114.070000
SHK3Dr    0.000000    5.635459
...            ...         ...
MPTS      0.000000   -0.000000
MOCOS     0.000000   -0.000000
BMOGDS2   0.000000   -0.000000
FESD2s    0.000000   -0.000000
OCTNLL    0.000000   -0.000000

[2712 rows x 2 columns]
